In [2]:
import tensorflow as tf
sess = tf.Session()
from keras import backend as K
K.set_session(sess)
from osgeo import ogr, osr

import numpy as np 

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.metrics import f1_score
import rasterio
from rasterio.transform import from_origin
from tqdm import tnrange, tqdm_notebook
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, constants
from scipy.ndimage import median_filter
%run ../../src/utils.py

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model = "../../models/april-27-ft/126-75-3/"
new_saver = tf.train.import_meta_graph(model + "model.meta")
new_saver.restore(sess, tf.train.latest_checkpoint(model))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../../models/april-27-ft/126-75-3/model


In [4]:
for i in range(50):
    try:
        fm = tf.get_default_graph().get_tensor_by_name("conv2d_{}/Sigmoid:0".format(i))
    except Exception:
        pass
#logits = tf.get_default_graph().get_tensor_by_name("conv2d_11/Sigmoid:0")
inp = tf.get_default_graph().get_tensor_by_name("Placeholder:0")
labels = tf.get_default_graph().get_tensor_by_name("Placeholder_2:0")
rmax = tf.get_default_graph().get_tensor_by_name("Placeholder_4:0")
rmin = tf.get_default_graph().get_tensor_by_name("Placeholder_5:0")
length = tf.get_default_graph().get_tensor_by_name("Placeholder_1:0")
dmax = tf.get_default_graph().get_tensor_by_name("Placeholder_6:0")

In [5]:
max_all = [0.20792677521705638,
 0.2990041905641556,
 0.4360648360848427,
 0.5161105132102968,
 0.4825860628485681,
 0.4943232241272928,
 0.5178957056999209,
 0.5291672283411026,
 0.6659183305501939,
 0.6092100739479065,
 0.37346625328063965,
 0.7162704998254776,
 0.40363759160041823,
 0.6478493613004686,
 0.8438000082969666,
 0.15365000069141388]

min_all = [0.01872335869818926,
 0.03467957343906164,
 0.021185510214418172,
 0.08898216009140014,
 0.052402322441339494,
 0.07628638863563539,
 0.083824477866292,
 0.08768215030431747,
 0.05718051139265299,
 0.0372315139323473,
 0.0,
 0.007564654648303981,
 -0.019136652257293465,
 0.0064204379683360435,
 0.006225000135600567,
 4.999999873689376e-05]

In [6]:
from skimage.transform import resize


test_x = np.load("../../tile_data/processed/test_x_l2a_processed.npy")
print(test_x.shape)
test_y = np.load("../../tile_data/processed/test_y_l2a_processed.npy")
test_lengths = np.load("../../tile_data/processed/test_length_l2a_processed.npy")

test_x = np.delete(test_x, 14, -1)

below_1 = [i for i, val in enumerate(test_x[:, :, :, :, :-2]) if np.min(val) < -1.5]
above_1 = [i for i, val in enumerate(test_x[:, :, :, :, :-2]) if np.max(val) > 1.5]
min_vals = [np.min(val) for i, val in enumerate(test_x[:, :, :, :, :-2]) if np.min(val) < -1.5]
max_vals = [np.max(val) for i, val in enumerate(test_x[:, :, :, :, :-2]) if np.max(val) > 1.5]
nans = [i for i, val in enumerate(test_x) if np.sum(np.isnan(val)) > 0]
outliers = below_1 + above_1 + nans
outliers = list(set(outliers))
print("There are {} outliers: {}".format(len(outliers), outliers))
print("\n")
print(min_vals, max_vals)
        
test_x = test_x[[x for x in range(0, len(test_x)) if x not in outliers]]
test_y = test_y[[x for x in range(0, len(test_y)) if x not in outliers]]
test_lengths = test_lengths[[x for x in range(0, len(test_lengths)) if x not in outliers]]

for sample in tnrange(0, len(test_x)):
    filtered = median_filter(test_x[sample, 0, :, :, 10], size = 5)
    filtered = np.reshape(filtered, (8, 2, 8, 2))
    filtered = np.mean(filtered, axis = (1, 3))
    filtered = resize(filtered, (16, 16), 0)
    test_x[sample, :, :, :, 10] = np.stack([filtered] * 24)
    
for band in range(0, test_x.shape[-1]):
    mins = min_all[band]
    maxs = max_all[band]
    test_x[:, :, :, :, band] = np.clip(test_x[:, :, :, :, band], mins, maxs)
    midrange = (maxs + mins) / 2
    rng = maxs - mins
    standardized = (test_x[:, :, :, :, band] - midrange) / (rng / 2)
    test_x[:, :, :, :, band] = standardized
    
    
print("The data has been scaled to [{}, {}]".format(np.min(test_x), np.max(test_x)))
print(test_x.shape)

(1111, 24, 16, 16, 17)
There are 22 outliers: [789, 1046, 794, 1053, 1054, 1056, 805, 1064, 682, 1069, 1071, 1082, 1084, 1087, 714, 593, 1105, 723, 1109, 750, 890, 764]


[-1.7179548, -1.5627671, -1.8532804, -1.7344598, -1.566039, -1.5506328, -1.5647379, -1.5482582, -1.5288317, -1.6640986, -1.5696894, -1.6236508, -1.6086293, -1.8026409, -1.6395152, -1.7323623, -1.8079871, -1.6267077] [1.6422757, 1.6066049, 1.7172077, 1.5512294, 1.5134994, 1.6136878, 1.644342, 1.5873938, 1.7966671, 1.7034032, 1.5901011, 1.7489257, 1.6376961, 1.6454076, 1.6791666, 1.7280316]



The data has been scaled to [-1.0000001192092896, 1.000000238418579]
(1089, 24, 16, 16, 16)


In [17]:
clipping_params = {
    'rmax': rmax,
    'rmin': rmin,
    'dmax': dmax
}


def run_predictions(x, y, lengths, debug = False):
    preds = []
    trues = []
    diffs = None
    for sample in tnrange(x.shape[0]):
        if sample not in [558, 563, 596, 599, 602, 615, 634]:
            pred = sess.run([fm], feed_dict={inp: x[sample].reshape(1, 24, 16, 16, 16),
                                      length: lengths[sample].reshape(1, 1),
                                      clipping_params['rmax']: 5,
                                      clipping_params['rmin']: 0,
                                      clipping_params['dmax']: 3,
                                      })
            preds.append(np.array(pred))
            trues.append(y[sample])
    if debug:
        thresh = 0.58
        preds_i = np.copy(preds)
        for i in range(len(preds_i)):
            preds_i[i] = np.array(preds_i[i])
            preds_i[i][np.where(preds_i[i] >= thresh)] = 1.
            preds_i[i][np.where(preds_i[i] < thresh)] = 0.
        diffs = [abs(np.sum(np.array(a)) - np.sum(np.array(b))) for a, b in zip(preds_i, trues)]
    #preds = np.concatenate(preds)
    #trues = np.concatenate(trues)
    return preds, trues, diffs

def compute_f1_score_at_tolerance(true, pred, tolerance = 1):
    
    tp = np.zeros_like(true)
    fp = np.zeros_like(true)
    fn = np.zeros_like(true)
    
    
    for x in range(true.shape[0]):
        for y in range(true.shape[1]):
            min_x = np.max([0, x-1])
            min_y = np.max([0, y-1])
            max_y = np.min([true.shape[0], y+2])
            max_x = np.min([true.shape[0], x+2])
            if true[x, y] == 1:
                if np.sum(pred[min_x:max_x, min_y:max_y]) > 0:
                    tp[x, y] = 1
                else:
                    fn[x, y] = 1
            if pred[x, y] == 1:
                if np.sum(true[min_x:max_x, min_y:max_y]) > 0:
                    if true[x, y] == 1:
                        tp[x, y] = 1
                else:
                    fp[x, y] = 1                
                
    return np.sum(tp), np.sum(fp), np.sum(fn)

def calc_thresh_relaxed(trues, preds):
    means = []
    for thresh in tnrange(10, 90, 2):
        tp = 0
        fp = 0 
        fn = 0
        errors = []
        for i in range(len(trues)):
            pred_c = np.copy(preds[i])
            pred_c = np.array(pred_c.reshape((14, 14)))
            pred_c[np.where(pred_c >= thresh / 100)] = 1.
            pred_c[np.where(pred_c < thresh / 100)] = 0.
            errors.append(abs(np.sum(trues[i]) - np.sum(pred_c)))
           
            
            tpi, fpi, fni = compute_f1_score_at_tolerance(np.array(trues[i].reshape((14, 14))),
                                                 pred_c)
            tp += tpi
            fp += fpi
            fn += fni
        ua = (tp) / (tp + fn)
        pa = (tp) / (tp + fp)
        errors = np.mean(errors)
        print(thresh, ua, pa, errors)
        #m#eans.append(abs[np.sum(trues[i]) - np.sum(pred_c)])
        means.append((ua + pa) / 2)
    
    best_acc = np.max(means)
    best_thresh = [x for x in range(10, 90, 2)][np.argmax(np.array(means))] / 100
    return best_thresh

def calc_thresh(trues, preds, threshold = None):
    trues = np.array(trues).flatten()
    preds = np.array(preds).flatten()
    f1s = []
    if not threshold:
        for thresh in range(10, 90, 2):
            pred_i = np.copy(preds)
            pred_i[np.where(pred_i >= thresh / 100)] = 1.
            pred_i[np.where(pred_i < thresh / 100)] = 0.
            f1 = f1_score(trues, pred_i)
            f1s.append(f1)
        best_f1 = np.max(f1s)
        best_thresh = [x for x in range(10, 90, 2)][np.argmax(np.array(f1s))] / 100
    else:
        pred_i = np.copy(preds)
        pred_i[np.where(pred_i >= threshold)] = 1.
        pred_i[np.where(pred_i < threshold)] = 0.
        f1 = f1_score(trues, pred_i)
        best_f1 = f1
        best_thresh = threshold
        
    return best_thresh

In [18]:
#preds, trues, diffs = run_predictions(test_x, test_y, test_lengths, debug = False)
thresh = calc_thresh_relaxed(trues, preds)
print(thresh)

10 0.9855021538541626 0.8232588984801374 31.51109057301294
12 0.9813766446726272 0.8544719040010879 26.92883548983364
14 0.9779929462902953 0.8761060915185077 23.842883548983362
16 0.9752729733598824 0.8919929058598074 21.557301293900185
18 0.9720584598966671 0.9029497098646034 19.910351201478743
20 0.96870078996343 0.9117232762949988 18.695009242144177
22 0.964887622171033 0.9191564801269495 17.684842883548985
24 0.9626621897734223 0.925770641168446 16.890942698706098
26 0.9598250888220825 0.9315649867374005 16.141404805914974
28 0.956844831400721 0.9363243890325128 15.571164510166358
30 0.9539556735511915 0.9408178457747202 15.073937153419593
32 0.9506110178425019 0.9445385540454916 14.746765249537892
34 0.947500618175666 0.9478706922365868 14.416820702402957
36 0.9439086922005753 0.9508882333661095 14.17837338262477
38 0.9400955244081781 0.9535224467705955 13.974121996303142
40 0.9360350863493798 0.9563344325071801 13.83826247689464
42 0.9329116724579966 0.9587908780846652 13.745841

In [ ]:
true_canopy = []
error_canopy = []
pred_canopy = []
tp_softs = []
fp_softs = []
fn_softs = []
for i in range(len(trues)):
    preds[i][np.where(preds[i] >= thresh)] = 1.
    preds[i][np.where(preds[i] < thresh)] = 0.
    
    true_canopy.append(np.sum(trues[i]) / 1.96)
    error_canopy.append(abs(np.sum(preds[i]) - np.sum(trues[i])) / 1.96)
    pred_canopy.append(np.sum(preds[i]) / 1.96)
    tp_soft, fp_soft, fn_soft = compute_f1_score_at_tolerance(np.array(trues[i].reshape((14, 14))),
                                                 np.array(preds[i].reshape((14, 14))))
    tp_softs.append(tp_soft)
    fp_softs.append(fp_soft)
    fn_softs.append(fn_soft)

In [1]:
print(np.mean(error_canopy))

NameError: name 'np' is not defined

In [14]:
precision = np.sum(tp_softs) / (np.sum(tp_softs) + np.sum(fp_softs))
recall = np.sum(tp_softs) / (np.sum(tp_softs) + np.sum(fn_softs))
print(precision, recall, np.mean(error_canopy), np.mean(true_canopy), np.mean(pred_canopy))

0.9760131210278138 0.9874853073359607 11.280612244897961 73.79081632653062 83.21428571428571


# Wall to wall